<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Aladin Lite HiPS Viewer — Interactive Exploration of ESO Surveys**

This notebook demonstrates an interactive astronomical image viewer powered by the [Aladin Lite](https://aladin.u-strasbg.fr/AladinLite/) widget. It is tailored for exploring **HiPS (Hierarchical Progressive Survey)** data directly within a Jupyter Notebook, including multi-wavelength overlays and real-time manipulation of the field of view.

---

### 🌍 Features

* **Target Search**: Quickly jump to any object by name (e.g., `NGC 4535`) or by coordinates.
* **Multi-Survey Image Viewer**: Choose between popular survey backgrounds (e.g., DSS2, SDSS, PanSTARRS, and ESO HiPS previews).
* **Overlay Options**: Visualize additional datasets such as:

  * Infrared (2MASS, WISE)
  * X-ray (XMM-Newton)
  * UV (GALEX)
  * Radio (VLASS)
  * JWST and Herschel imaging
* **Opacity & Zoom Controls**: Fine-tune the appearance of overlays and field-of-view (FOV).
* **ESO Integration**: Display previews from ESO’s HiPS service (when a `dp_id` is known).

---

### 🎯 Use Case

This viewer is particularly suited for:

* **Quick visual inspection** of data coverage and context
* **Multi-wavelength comparison** for a given field or object
* **Educational outreach** or interactive tutorials
* Supporting data exploration from large ESO programs or public surveys

---

### 🧰 Requirements

Make sure to install the following packages:

```bash
pip install ipywidgets ipyaladin
```

Enable widgets in your notebook interface:

```bash
jupyter nbextension enable --py widgetsnbextension
```

---

### 📌 Tip

You can easily extend this notebook by:

* Adding marker overlays or catalog layers (e.g., SIMBAD, VizieR)
* Integrating direct queries from the ESO archive using `astroquery.eso`
* Exporting snapshots or sharing interactive notebooks

<hr style="border:2px solid #0281c9"> </hr>

In [1]:
import astroquery # import astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

astroquery version: 0.4.11.dev10245


In [2]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [3]:
eso = Eso() # create an instance of the ESO class 

In [4]:
eso.maxrec = 100    # For this example we limit the number of records to 3

# **Performing a Small Cone Search**

In [5]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

target = "NGC4535" # set the target 
coords = SkyCoord.from_name(target) # create a SkyCoord object from the name of the source 
radius = 2 *u.arcsec # set the radius of the search 

table_reduced = eso.query_surveys("PHANGS",
                                cone_ra=coords.ra.value, 
                                cone_dec=coords.dec.value, 
                                cone_radius=radius.to("deg").value) # query the ESO archive for HAWKI data

table_reduced["target_name", "s_ra", "s_dec", "proposal_id", "instrument_name", "dp_id", "release_description"][:3] # print the first 3 rows of the table
table_reduced

target_name,s_ra,s_dec,dp_id,proposal_id,abmaglim,access_estsize,access_format,access_url,bib_reference,calib_level,dataproduct_subtype,dataproduct_type,em_max,em_min,em_res_power,em_xel,facility_name,filter,gal_lat,gal_lon,instrument_name,last_mod_date,multi_ob,n_obs,o_calib_status,o_ucd,obs_collection,obs_creator_did,obs_creator_name,obs_id,obs_publisher_did,obs_release_date,obs_title,obstech,p3orig,pol_states,pol_xel,preview_html,publication_date,release_description,s_fov,s_pixel_scale,s_region,s_resolution,s_xel1,s_xel2,snr,strehl,t_exptime,t_max,t_min,t_resolution,t_xel
,deg,deg,,,mag,kbyte,,,,,,,m,m,,,,,deg,deg,,,,,,,,,,,,,,,,,,,,,deg,arcsec,,arcsec,,,,,s,d,d,s,
object,float64,float64,object,object,float64,int64,object,object,object,int32,object,object,float64,float64,float64,int64,object,object,float64,float64,object,object,object,int32,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,float64,float64,object,float64,int64,int64,float64,float64,float64,float64,float64,float64,int64
NGC4535_Pri01,188.576744,8.19195,ADP.2021-07-16T10:20:56.513,1100.B-0651(B),0.0,18541840,application/x-votable+xml;content=datalink,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2021-07-16T10:20:56.513,,3,ifs,cube,9.3475e-07,4.7025e-07,2830.0,3761,ESO-VLT-U4,,70.634098,290.052019,MUSE,2022-11-21T12:47:36.693Z,M,24,absolute,,PHANGS,ivo://eso.org/origfile?NGC4535_PHANGS_DATACUBE_copt_0.56asec.fits,"SCHINNERER, E.","1989493, 1989498, 1989502, 1989506, 1989510, 1989514",ivo://eso.org/ID?ADP.2021-07-16T10:20:56.513,2021-07-16T11:12:31Z,,IFU,EDP,,--,https://archive.eso.org/dataset/ADP.2021-07-16T10:20:56.513,2021-07-16T15:16:57Z,http://www.eso.org/rm/api/v1/public/releaseDescriptions/184,0.06031044666,0.2,UNION J2000 (POLYGON J2000 188.602045 8.167384 188.567809 8.167385 188.567808 8.217274 188.602048 8.217273 POLYGON J2000 188.602045 8.167384 188.567809 8.167385 188.567808 8.217274 188.602048 8.217273),0.56,1220,1220,--,--,2580.0,58255.0399384,58217.13771896,3274751.759616,--
NGC4535_Pri01,188.576744,8.19195,ADP.2021-07-16T10:20:56.519,1100.B-0651(B),0.0,18541837,application/x-votable+xml;content=datalink,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2021-07-16T10:20:56.519,,3,ifs,cube,9.35e-07,4.7e-07,2830.0,3761,ESO-VLT-U4,,70.634098,290.052019,MUSE,2022-11-21T12:47:36.770Z,M,24,absolute,,PHANGS,ivo://eso.org/origfile?NGC4535_PHANGS_DATACUBE_native.fits,"SCHINNERER, E.","1989493, 1989498, 1989502, 1989506, 1989510, 1989514",ivo://eso.org/ID?ADP.2021-07-16T10:20:56.519,2021-07-16T12:12:31Z,,IFU,EDP,,--,https://archive.eso.org/dataset/ADP.2021-07-16T10:20:56.519,2021-07-16T15:16:57Z,http://www.eso.org/rm/api/v1/public/releaseDescriptions/184,0.06031044666,0.2,UNION J2000 (POLYGON J2000 188.602045 8.167384 188.567809 8.167385 188.567808 8.217274 188.602048 8.217273 POLYGON J2000 188.602045 8.167384 188.567809 8.167385 188.567808 8.217274 188.602048 8.217273),0.433,1220,1220,--,--,2580.0,58255.0399384,58217.13771896,3274751.759616,--
NGC4535_Pri01,188.576744,8.19195,ADP.2025-05-14T09:12:30.243,1100.B-0651(B),0.0,1203,application/x-votable+xml;content=datalink,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2025-05-14T09:12:30.243,2023MNRAS.520.4902G,4,catalogtile,measurements,9.3475e-07,4.7025e-07,1.5123789310455322,--,ESO-VLT-U4,,70.634098,290.052018,MUSE,2025-05-14T11:02:40.963Z,M,1,,,PHANGS,ivo://eso.org/origfile?NGC4535_catalog.fits,"SCHINNERER, E.","1989493, 1989498, 1989502, 1989506, 1989510, 1989514",ivo://eso.org/ID?ADP.2025-05-14T09:12:30.243,2025-05-14T10:58:26Z,,IFU,EDP,,--,https://archive.eso.org/dataset/ADP.2025-05-14T09:12:30.243,2025-05-14T10:58:27Z,http://www.eso.org/rm/api/v1/public/releaseDescriptions/235,0.06023380944,--,POLYGON J2000 188.602017 8.167412 188.567837 8.167412 188.567835 8.217246 188.60202 8.217245,--,--,--,--,--,--,58255.03993839,58217.13771896,3274751.758752,--


## **Aladin Preview**

In [6]:
from ipyaladin import Aladin # Aladin Lite widget for Jupyter notebooks

### **Simple Aladin Lite Viewer**

Here we show a simple Aladin Lite viewer covering the source with the default survey image (DSS2 color).

In [7]:
aladin = Aladin(fov=0.5, # Field of view in degrees
                target=target, # Target to display
                ) 
aladin

Aladin()

### **Simple Aladin Lite HiPS Viewer**

Here we show a simple Aladin Lite viewer covering the source but now showing the ESO HiPS preview image (white-light image). 

In [8]:
dp_id = table_reduced["dp_id"][0] # get the first data product ID from the table
hips_url = f"https://archive.eso.org/previews/v1/files/{dp_id}/hips" 

aladin = Aladin(fov=0.5, # Field of view in degrees
                target=target, # Target to display
                survey=hips_url, # HIPS URL to display
                )
aladin

Aladin(survey='https://archive.eso.org/previews/v1/files/ADP.2021-07-16T10:20:56.513/hips')

### **Simple Aladin Lite Viewer with Linked widgets**

Here we show a simple Aladin Lite viewer covering the source but now showing the ESO HiPS preview image (white-light image). The viewer is linked to compare between different surveys. Try moving one of the views and the others will follow!

In [9]:
def show_aladin_linked(target, fov=0.025, dp_id=None):
    """
    Display Aladin Lite views from multiple surveys side-by-side, synchronized in target and field-of-view (FoV).

    By default, four standard public surveys are shown. If a DP-ID is provided, a fifth panel is added
    showing the ESO HiPS preview for that Phase 3 product.

    Parameters
    ----------
    target : str
        The sky position (object name or coordinates) to center all viewers on.
    fov : float, optional
        Field of view in degrees (default: 0.025).
    dp_id : str or None, optional
        If provided, adds a fifth panel with the ESO HiPS preview (e.g. white-light image) for the given Phase 3 DP-ID.

    Returns
    -------
    ipywidgets.Box
        A horizontal widget containing the synchronized Aladin viewers.

    Notes
    -----
    - All viewers are synchronized in both target and zoom level.
    - The layout automatically adjusts panel widths based on the number of surveys.
    - DP-ID is the unique ESO file identifier (e.g. 'ADP.2017-09-19T14:57:26.141').
    """
    from ipyaladin import Aladin
    from ipywidgets import Layout, Box, widgets

    # Default surveys
    surveys = [
        "P/DSS2/color",
        "P/SDSS9/color-alt",
        "P/PanSTARRS/DR1/color-i-r-g",
        "P/PanSTARRS/DR1/color-z-zg-g"
    ]

    if dp_id:
        hips_url = f"https://archive.eso.org/previews/v1/files/{dp_id}/hips" 
        surveys.append(hips_url)

    width_percent = f"{100 // len(surveys)}%"

    cosmetic_options = {
        "show_projection_control": False,
        "show_fullscreen_control": False,
        "show_zoom_control": False,
        "show_share_control": False,
        "show_simbad_pointer_control": False,
        "show_coo_grid_control": False,
        "show_settings_control": False,
        "show_context_menu": False,
        "reticle": False
    }
    
    viewers = [
        Aladin(layout=Layout(width=width_percent), target=target, survey=survey, fov=fov, **cosmetic_options)
        for survey in surveys
    ]

    # Link targets and FoVs between all viewers
    for i in range(len(viewers) - 1):
        widgets.jslink((viewers[i], "_target"), (viewers[i + 1], "_target"))
        widgets.jslink((viewers[i], "_fov"), (viewers[i + 1], "_fov"))

    box_layout = Layout(display="flex", flex_flow="row", align_items="stretch", border="solid", width="100%")
    return Box(children=viewers, layout=box_layout)

show_aladin_linked(target, dp_id=table_reduced["dp_id"][0], fov=0.1)

Box(children=(Aladin(layout=Layout(width='20%'), survey='P/DSS2/color'), Aladin(layout=Layout(width='20%'), su…

In [10]:
def show_aladin_widgets(target, fov=0.025, dp_id=None):

    import ipywidgets as widgets
    import time

    hips_url = f"https://archive.eso.org/previews/v1/files/{dp_id}/hips" 

    aladin = Aladin(fov=fov, # Field of view in degrees
                    target=target, # Target to display
                    survey=hips_url, # HIPS URL to display
                    )

    survey_selector = widgets.ToggleButtons(
        options=[
            ("ESO HiPS preview", hips_url),
            ("DSS2 color", "P/DSS2/color"),
            ("SDSS9 color-alt", "P/SDSS9/color-alt"),
            ("PanSTARRS i-r-g", "P/PanSTARRS/DR1/color-i-r-g"),
            ("PanSTARRS z-zg-g", "P/PanSTARRS/DR1/color-z-zg-g"),
            ("2MASS color", "P/2MASS/color"),
            ("Chandra color", "P/cda/hips/allsky/rgb"),
            ("GALEX color", "P/GALEXGR6_7/color"),
            ("DESI Legacy DR10", "CDS/P/DESI-Legacy-Surveys/DR10/color"),
            ("DECaLS DR5", "CDS/P/DECaLS/DR5/color"),
            ("Rubin First Look", "CDS/P/Rubin/FirstLook"),
            ("Herschel PACS", "ESAVO/P/HERSCHEL/PACS_RGB_norm"),
            ("WISE color", "CDS/P/allWISE/color"),
            ("VLASS Quicklook", "NRAO/P/VLASS-Quicklook-MedianStack"),
            ("JWST MIRI Imaging", "ESAVO/P/JWST/P/MIRI_Imaging"),
            ("JWST NIRCam Imaging", "ESAVO/P/JWST/NIRCam_Imaging"),
        ],
        description="Image:",
        disabled=False,
        tooltips=[
            "ESO HiPS preview",
            "DSS2 color",
            "SDSS9 color-alt",
            "PanSTARRS i-r-g composite",
            "PanSTARRS z-zg-g composite",
            "2MASS near-infrared imaging",
            "Chandra color X-ray imaging",
            "GALEX ultraviolet imaging",
            "DESI Legacy Imaging Survey (DR10)",
            "Dark Energy Camera Legacy Survey (DECaLS DR5)",
            "Rubin Observatory First Look",
            "Herschel PACS RGB normalized map",
            "WISE mid-infrared survey",
            "VLA Sky Survey (VLASS) median stack",
            "JWST MIRI imaging preview",
            "JWST NIRCam imaging preview",
        ]
    )


    def on_survey_value_change(change: dict) -> None:
        """Survey change callback.

        Parameters
        ----------
        change : dict
            The change dictionary.
        """
        aladin.survey = change["new"]


    survey_selector.observe(on_survey_value_change, names="value")


    survey_overlay_selector = widgets.ToggleButtons(
        options=[
            ("ESO HiPS preview", hips_url),
            ("DSS2 color", "P/DSS2/color"),
            ("SDSS9 color-alt", "P/SDSS9/color-alt"),
            ("PanSTARRS i-r-g", "P/PanSTARRS/DR1/color-i-r-g"),
            ("PanSTARRS z-zg-g", "P/PanSTARRS/DR1/color-z-zg-g"),
            ("2MASS color", "P/2MASS/color"),
            ("Chandra color", "P/cda/hips/allsky/rgb"),
            ("GALEX color", "P/GALEXGR6_7/color"),
            ("DESI Legacy DR10", "CDS/P/DESI-Legacy-Surveys/DR10/color"),
            ("DECaLS DR5", "CDS/P/DECaLS/DR5/color"),
            ("Rubin First Look", "CDS/P/Rubin/FirstLook"),
            ("Herschel PACS", "ESAVO/P/HERSCHEL/PACS_RGB_norm"),
            ("WISE color", "CDS/P/allWISE/color"),
            ("VLASS Quicklook", "NRAO/P/VLASS-Quicklook-MedianStack"),
            ("JWST MIRI Imaging", "ESAVO/P/JWST/P/MIRI_Imaging"),
            ("JWST NIRCam Imaging", "ESAVO/P/JWST/NIRCam_Imaging"),
        ],
        description="Overlay:",
        disabled=False,
        tooltips=[
            "ESO HiPS preview",
            "DSS2 color",
            "SDSS9 color-alt",
            "PanSTARRS i-r-g composite",
            "PanSTARRS z-zg-g composite",
            "2MASS near-infrared imaging",
            "Chandra color X-ray imaging",
            "GALEX ultraviolet imaging",
            "DESI Legacy Imaging Survey (DR10)",
            "Dark Energy Camera Legacy Survey (DECaLS DR5)",
            "Rubin Observatory First Look",
            "Herschel PACS RGB normalized map",
            "WISE mid-infrared survey",
            "VLA Sky Survey (VLASS) median stack",
            "JWST MIRI imaging preview",
            "JWST NIRCam imaging preview",
        ]
    )

    def on_survey_overlay_value_change(change: dict) -> None:
        """Survey overlay change callback.

        Parameters
        ----------
        change : dict
            The change dictionary.
        """
        aladin.overlay_survey = change["new"]
        aladin.overlay_survey_opacity = aladin.overlay_survey_opacity + 0.00000001


    survey_overlay_selector.observe(on_survey_overlay_value_change, names="value")

    opacity_slider = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.01,
        description="Opacity:",
        disabled=False,
        continuous_update=True,
        orientation="horizontal",
        readout=False,
        readout_format=".1f",
    )


    def on_surveyoverlay_opacity_value_change(change: dict) -> None:
        """Survey overlay opacity change callback.

        Parameters
        ----------
        change : dict
            The change dictionary.
        """
        aladin.overlay_survey_opacity = change["new"]


    opacity_slider.observe(on_surveyoverlay_opacity_value_change, names="value")


    zoom_slider = widgets.FloatSlider(
        value=180 / aladin.fov.deg,
        min=1,
        max=400,
        step=1,
        description="Zoom:",
        disabled=False,
        continuous_update=True,
        orientation="horizontal",
        readout=False,
        readout_format=".1f",
    )


    def on_zoom_slider_value_change(change: dict) -> None:
        """Zoom slider change callback.

        Parameters
        ----------
        change : dict
            The change dictionary.
        """
        aladin.fov = 180 / change["new"]


    zoom_slider.observe(on_zoom_slider_value_change, names="value")

    # --- Screenshot Button ---
    screenshot_button = widgets.Button(
        description="Screenshot",
        icon="camera",
        tooltip="Download view as PNG",
        layout=widgets.Layout(width="120px")
    )

    def take_screenshot(_):
        img = aladin.save_view_as_image(path=f'./aladin_screenshot_{int(time.time())}.png') #screenshot with timestamp
        display(img)

    screenshot_button.on_click(take_screenshot)

    # --- Catalog Query (SIMBAD) ---
    catalog_input = widgets.Text(
        description="SIMBAD:",
        placeholder="e.g. M31",
        style={'description_width': 'initial'}
    )
    catalog_button = widgets.Button(
        description="Query",
        icon="database",
        layout=widgets.Layout(width="80px")
    )
    catalog_output = widgets.Output()

    def search_simbad(_):
        from astroquery.simbad import Simbad
        with catalog_output:
            catalog_output.clear_output()
            result = Simbad.query_object(catalog_input.value)
            display(result if result is not None else "No results.")

    catalog_button.on_click(search_simbad)

    vbox = widgets.VBox(
        [aladin, survey_selector, survey_overlay_selector, widgets.HBox([screenshot_button, opacity_slider, zoom_slider]),  widgets.HBox([catalog_input, catalog_button]), catalog_output]
    )

    return vbox

show_aladin_widgets(target, dp_id=table_reduced["dp_id"][0], fov=0.2)